# Лабораторная работа №4

### Выполнил Рыбаков Владислав, БПМ201, 30 вариант

Задания 5.1.30, 5.2, 5.6.8

### Задание 5.1.30

Дана система уравнений $Ax=b$. Найти решение системы с помощью метода Гаусса. Выполнить
10 итераций по методу Зейделя. Принимая решение, полученное с помощью метода Гаусса за точное, найти
величину абсолютной погрешности итерационного решения.

In [58]:
import numpy as np
from scipy import optimize

In [59]:
A = np.array([[0.33, 0.1, 0.1, 0, 0.02, 0.1],
                [0.99, 4.9, 0.4, 2.97, 0.21, -0.3],
                [1.32, -1.6, 6.6, 3.3, 0.24, 0.1],
                [1.98, 1.2, 1.1, 6.93, 0.81, -1.2],
                [1.98, -1.5, 0.4, -1.98, 6.1, 0],
                [0.99, 0.4, 0.3, 1.65, 0.9, 4.3]])
b = np.array([0.94, 18.68, 12.50, 5.56, -10.28, 12.29])
A

array([[ 0.33,  0.1 ,  0.1 ,  0.  ,  0.02,  0.1 ],
       [ 0.99,  4.9 ,  0.4 ,  2.97,  0.21, -0.3 ],
       [ 1.32, -1.6 ,  6.6 ,  3.3 ,  0.24,  0.1 ],
       [ 1.98,  1.2 ,  1.1 ,  6.93,  0.81, -1.2 ],
       [ 1.98, -1.5 ,  0.4 , -1.98,  6.1 ,  0.  ],
       [ 0.99,  0.4 ,  0.3 ,  1.65,  0.9 ,  4.3 ]])

In [60]:
# решим систему
true_solve = np.linalg.solve(A, b)
print(true_solve)

[ 0.3030303   3.6         2.6         0.21212121 -1.          2.4       ]


In [62]:
B = np.zeros((6,6))
for i in range(6):
    for j in range(6):
        if i != j:
            B[i,j] = - A[i,j]/A[i,i]
            
B

array([[ 0.        , -0.3030303 , -0.3030303 , -0.        , -0.06060606,
        -0.3030303 ],
       [-0.20204082,  0.        , -0.08163265, -0.60612245, -0.04285714,
         0.06122449],
       [-0.2       ,  0.24242424,  0.        , -0.5       , -0.03636364,
        -0.01515152],
       [-0.28571429, -0.17316017, -0.15873016,  0.        , -0.11688312,
         0.17316017],
       [-0.32459016,  0.24590164, -0.06557377,  0.32459016,  0.        ,
        -0.        ],
       [-0.23023256, -0.09302326, -0.06976744, -0.38372093, -0.20930233,
         0.        ]])

In [63]:
# проверим достаточное условие сходимости
max_norm = np.linalg.norm(B, ord=np.inf)
max_norm
# <1, следовательно, метод сходится

0.993939393939394

In [114]:
def zeidel(A, b, x0):
    x = x0
    count = 0
    for k in range(10):
        x_new = np.copy(x)
        for i in range(6): 
            B1 = sum(-A[i][j]/A[i][i] * x_new[j] for j in range(i))
            B2 = sum(-A[i][j]/A[i][i] * x[j] for j in range(i + 1, 6))
            x_new[i] = b[i]/A[i][i] + B1 + B2
        x = x_new
    return x

In [115]:
x0 = np.zeros(6)
zeidel_solve = zeidel(A, b, x0)
print("Ответ:", zeidel_solve)
print("Погрешность:", np.max(np.abs(true_solve - zeidel_solve)))

Ответ: [ 0.30302158  3.59997928  2.59997792  0.21212597 -0.99999927  2.4000035 ]
Погрешность: 2.2084962623125648e-05


In [117]:
# попробуем другое начальное приближение
x0 = np.zeros(6)
x0 += 0.5
zeidel_solve = zeidel(A, b, x0)
print("Ответ:", zeidel_solve)
print("Погрешность:", np.max(np.abs(true_solve - zeidel_solve)))

Ответ: [ 0.3030178   3.59998406  2.59998393  0.21212582 -0.99999731  2.40000315]
Погрешность: 1.607173615836288e-05


**Получаем, что метод Зейделя позволяет получить решение системы линейных уравнений и от выбора начального приближения зависит полученный результат. В целом, даже при 10 итерациях, метод показывает очень хорошую точность.**

### Задание 5.2

Для системы уравнений $Ax = b$ из задачи 5.1 найти решение по методу Зейделя с точностью $\varepsilon = 10^{−6}$, взяв любое начальное приближение. Для этого модифицировать программу из пункта 3 так, чтобы решение вычислялось с заданной точностью $\varepsilon$. Предусмотреть подсчет количества итераций, потребовавшихся для достижения точности $\varepsilon$.

In [118]:
# изменим функцию Зейделя, чтобы она подсчитывала количество итераций
def zeidel_new(A, b, eps):
    n = len(A)
    iter_num = 0
    x = [0. for i in range(n)]
    accuracy = True
    while accuracy:
        iter_num += 1
        x_new = np.copy(x)
        for i in range(n): 
            B1 = sum(-A[i][j]/A[i][i] * x_new[j] for j in range(i))
            B2 = sum(-A[i][j]/A[i][i] * x[j] for j in range(i + 1, n))
            x_new[i] = b[i]/A[i][i] + B1 + B2
        accuracy = np.max(np.abs(x_new - x)) > eps
        x = x_new
    return x, iter_num

In [119]:
solve, iter_num = zeidel_new(A, b, 10**(-6))
print("Ответ:", solve)
print("Итераций:", iter_num)

Ответ: [ 0.30303002  3.60000003  2.60000007  0.21212125 -0.99999989  2.40000002]
Итераций: 14


### Задание 5.6.8

Дана система уравнений $Ax=b$, где $A$ – симметричная положительно определенная разреженная*
матрица размерности $n * n$. Методом Зейделя найти решение системы с точностью $\varepsilon = 10^{-9}$. Определить
число итераций, потребовавшихся для достижения заданной точности.

на главной диагонали элементы равны 450, на первой
наддиагонали элементы равны 55, на 7 наддиагонали
элементы равны 7, на 8
наддиагонали равны 2

In [188]:
# n = 10 # для проверки правильности создания матрицы
n = 40
A = np.zeros((n,n))
for i in range(n):
    A[i,i] = 450
    
    A[i,i+1:i+2] = 55
    A[i,i-1:i] = 55
    
    A[i,i-8:i] = 2
    A[i,i-7:i] = 0
    
    A[i,i+7:i+8] = 7
    A[i,i-7:i] = 7
    A[i,i-6:i] = 0
    
    A[i,i+8:i+9] = 2
A

array([[450.,  55.,   0., ...,   0.,   0.,   0.],
       [ 55., 450.,  55., ...,   0.,   0.,   0.],
       [  0.,  55., 450., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 450.,  55.,   0.],
       [  0.,   0.,   0., ...,   0., 450.,  55.],
       [  0.,   0.,   0., ...,   0.,   0., 450.]])

In [193]:
b = np.zeros(n)
for i in range(n):
    b[i] = (i+1)*(i+1)*np.e**(5/(i+1))
b

array([ 148.4131591 ,   48.72997584,   47.65041045,   55.84548732,
         67.95704571,   82.83513207,  100.09362644,  119.56774128,
        141.17562889,  164.87212707,  190.63030951,  218.43313868,
        248.26931376,  280.13104635,  314.01279564,  349.91051294,
        387.8211715 ,  427.74246345,  469.67259852,  513.61016668,
        559.55404236,  607.50331621,  657.45724558,  709.41521799,
        763.37672385,  819.34133571,  877.30869238,  937.27848656,
        999.25045514, 1063.22437158, 1129.20003972, 1197.17728888,
       1267.15596986, 1339.13595165, 1413.11711875, 1489.09936893,
       1567.08261139, 1647.06676521, 1729.05175801, 1813.03752491])

In [194]:
solve, iter_num = zeidel_new(A, b, 10**(-9))
print("Ответ:", solve)
print("Итераций:", iter_num)

Ответ: [0.3186827  0.05394981 0.08160622 0.09280097 0.11440099 0.1378984
 0.18513752 0.21680737 0.2605272  0.30649144 0.3554402  0.4082863
 0.4650647  0.52539108 0.58979998 0.65802513 0.73008587 0.80601146
 0.88578791 0.969411   1.05685993 1.14814126 1.2432546  1.34219721
 1.44501189 1.55141279 1.66083094 1.77503857 1.89308486 2.01488516
 2.14092876 2.26883332 2.40479144 2.60690083 2.75094432 2.89900441
 3.04992206 3.21322399 3.31090362 3.98148071]
Итераций: 12
